In [2]:
import findspark
findspark.init('/home/seanzhen52/spark-2.2.3-bin-hadoop2.7/')

In [3]:
cat word.txt

how are you
hello world
how dare you
fuck you motherfucker
lol
read me please.


In [5]:
from pyspark import SparkContext
sc = SparkContext('local','wordcount')

In [6]:
textFile = sc.textFile('file:///home/seanzhen52/word.txt')

In [7]:
textFile.first()

'how are you'

In [9]:
#惰性机制
textFile1 = sc.textFile("file:///usr/local/spark/mycode/wordcount/word123.txt")

In [10]:
textFile1.first()

Py4JJavaError: An error occurred while calling o46.partitions.
: org.apache.hadoop.mapred.InvalidInputException: Input path does not exist: file:/usr/local/spark/mycode/wordcount/word123.txt
	at org.apache.hadoop.mapred.FileInputFormat.singleThreadedListStatus(FileInputFormat.java:287)
	at org.apache.hadoop.mapred.FileInputFormat.listStatus(FileInputFormat.java:229)
	at org.apache.hadoop.mapred.FileInputFormat.getSplits(FileInputFormat.java:315)
	at org.apache.spark.rdd.HadoopRDD.getPartitions(HadoopRDD.scala:199)
	at org.apache.spark.rdd.RDD$$anonfun$partitions$2.apply(RDD.scala:252)
	at org.apache.spark.rdd.RDD$$anonfun$partitions$2.apply(RDD.scala:250)
	at scala.Option.getOrElse(Option.scala:121)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:250)
	at org.apache.spark.rdd.MapPartitionsRDD.getPartitions(MapPartitionsRDD.scala:46)
	at org.apache.spark.rdd.RDD$$anonfun$partitions$2.apply(RDD.scala:252)
	at org.apache.spark.rdd.RDD$$anonfun$partitions$2.apply(RDD.scala:250)
	at scala

In [16]:
#
textFile.saveAsTextFile('file:///home/seanzhen52/writeback')#saveAsTextFile()括号里面的参数是保存文件的路径，不是文件名

saveAsTextFile()是一个“行动”（Action）类型的操作，所以，马上会执行真正的计算过程，从word.txt中加载数据到变量textFile中，然后，又把textFile中的数据写回到本地文件目录“/home/seanzhen52/writeback/”下面

In [17]:
ls

anaconda3/                       spark-2.2.3-bin-hadoop2.7.tgz
Anaconda3-5.0.0-Linux-x86_64.sh  spark-warehouse/
donald.json                      test.py
first pyspark.ipynb              tweets.json
hillary.json                     tweets.json.1
hs_err_pid2511.log               word2vecM_simple/
hs_err_pid2996.log               word2vecM_simple.zip
hs_err_pid3589.log               WordCount.ipynb
hs_err_pid3741.log               word.txt
hs_err_pid7525.log               writeback/
spark-2.2.3-bin-hadoop2.7/


In [21]:
cd writeback/

/home/seanzhen52/writeback


In [22]:
ls

part-00000  _SUCCESS


In [23]:
cat part-00000 _SUCCESS

how are you
hello world
how dare you
fuck you motherfucker
lol
read me please.


In [24]:
cd .. 

/home/seanzhen52


In [27]:
wordCount = textFile.flatMap(lambda line:line.split(" ")).map(lambda word:(word,1)).reduceByKey(lambda a,b:a+b)

In [28]:
wordCount.collect()

[('how', 2),
 ('are', 1),
 ('you', 3),
 ('hello', 1),
 ('world', 1),
 ('dare', 1),
 ('fuck', 1),
 ('motherfucker', 1),
 ('lol', 1),
 ('read', 1),
 ('me', 1),
 ('please.', 1)]

In [29]:
a = wordCount.collect()

In [31]:
type(a)

list

textFile.flatMap(labmda line : line.split(” “))会遍历textFile中的每行文本内容，当遍历到其中一行文本内容时，会把文本内容赋值给变量line，并执行Lamda表达式line : line.split(” “)。

- 然后执行map()操作，也就是map(lambda word : (word, 1))，这个map操作会遍历这个集合中的每个单词，当遍历到其中一个单词时，就把当前这个单词赋值给变量word，并执行Lamda表达式word : (word, 1)
- 这个Lamda表达式的含义是，word作为函数的输入参数，然后，执行函数处理逻辑，这里会执行(word, 1)，也就是针对输入的word，构建得到一个tuple，形式为(word,1)，key是word，value是1（表示该单词出现1次）。

- 已经得到一个RDD，这个RDD的每个元素是(key,value)形式的tuple。最后，针对这个RDD，执行reduceByKey(labmda a, b : a + b)操作，这个操作会把所有RDD元素按照key进行分组，然后使用给定的函数（这里就是Lamda表达式：a, b : a + b），
- 对具有相同的key的多个value进行reduce操作，返回reduce后的(key,value)，比如(“hadoop”,1)和(“hadoop”,1)，具有相同的key，进行reduce以后就得到(“hadoop”,2)，这样就计算得到了这个单词的词频